Use continuous geographic, hydrological, climate, and meteorological datasets as input to train neural network(NN) and random forest(RF) models, to determine the major environmentl drivers of algae growth in lakes.
Use Google Cloud Platform (GCP) to extract relative data from its [Earth Engine Dataset](https://developers.google.com/earth-engine/datasets) and run further NN models

## 1. Climate data preparation from Google Earth Engine

In [1]:
##connect to google earth engine
import ee
service_account = '429406912917-compute@developer.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'credentials.json')
ee.Initialize(credentials)

In [86]:
##prepare a mapping tool
import folium
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [88]:
### extract climate data from ECMWF ERA 5
##details of dataset: https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_DAILY#description
## https://cds.climate.copernicus.eu/cdsapp#!/software/app-era5-explorer?tab=overview
sensor='ECMWF/ERA5/DAILY'
start_date,end_date='2020-06-01','2020-07-01'
aoi =ee.Geometry.Rectangle(-95.4, 48.88,-93.75, 49.77)
era5 = ee.ImageCollection(sensor) \
       .filterBounds(aoi)  \
       .filterDate(ee.Date(start_date), ee.Date(end_date))
era5_2mt = era5.select('mean_2m_air_temperature')
print('Number of images', era5_2mt.size().getInfo())

Number of images 30


In [122]:
##preview the average imagery
avgT=era5_2mt.mosaic().add(-273.15)  #kelvin to Celsius  c=k-273.15   #.clip(aoi)
##result = ee.Image.expression('(slope * aspect) / 255', {'slope': slope,'aspect': aspect})
print("Global temperature Jun 2020 monthly avg (C)")

## #show interactive map
## vis_params = {
##   'min': -30,
##   'max': 40,
##   'palette': ['blue', 'green', 'red']
## }
## center = aoi.centroid()
## map = folium.Map(location=[center.coordinates().get(1).getInfo(),center.coordinates().get(0).getInfo()], zoom_start=5)
## #map = folium.Map(location=[48.88,-95.4], zoom_start=10)
## # Add the image layer to the map and display it.
## map.add_ee_layer(avgT, vis_params, 'Avg Temp (c)')
## map.add_child(folium.LayerControl())
## display(map)

##display a raster imagery
from IPython.display import Image
Image(url =avgT.updateMask(avgT.gt(-10)).getThumbURL({'min':0, 'max': 40, 'dimensions': 512,
                'palette': ['blue', 'green', 'red']}))

Global temperature Jun 2020 monthly avg (C)


In [6]:
## export to a test imagery to Google Cloud Platform (GCP) buckets 
img=era5_2mt.first()
export_config = {
    'image': img,
    'region': aoi,
    'crs': 'EPSG:4326',
    'scale': 100000,    
    'fileFormat': 'GeoTIFF',
    'fileNamePrefix': 'ERA5_2mt',
    'bucket':   'AlgaeDrive',  
    'maxPixels':1e10,
    'description': 'ERA5 climate data download: ER5 mean_2m_air_temperature'
}
task=ee.batch.Export.image.toCloudStorage(**export_config)
#task=ee.batch.Export.image.toDrive(**export_config)
task.start()
import time
time.sleep(10)
print(task.status())

{'state': 'READY', 'description': 'ERA5 climate data download: ER5 mean_2m_air_temperature', 'creation_timestamp_ms': 1680028182464, 'update_timestamp_ms': 1680028182464, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'UQFQZPCG5XE5SKJFP2QS4TKZ', 'name': 'projects/earthengine-legacy/operations/UQFQZPCG5XE5SKJFP2QS4TKZ'}


In [137]:
##iterate all the image to export to GCP bucket
def export_ERA5_to_GCPbucket(era5_dataset,aoi=ee.Geometry.Rectangle(-95.4, 48.88,-93.75, 49.77), bucket_name='AlgaeDrive',crs_epsg=4326,scale=1000,prdct_key=None):
  img_list = era5_dataset.toList(era5_dataset.size())
  nImages = img_list.size().getInfo()
  for i in range(nImages):
    img=ee.Image(img_list.get(i)).clip(aoi)
    product_id=img.getInfo()['id'].replace('/','_')
    export_config = {
      'image': img.float(),
      'region': aoi,
      'crs': f'EPSG:{crs_epsg}',
      'scale': scale,    
      'fileFormat': 'GeoTIFF',
      'fileNamePrefix': f'ERA5/{product_id}_{prdct_key}',
      'bucket':   bucket_name,  
      'maxPixels':1e10,
      'description': f'ERA5 climate data download: ER5 product: {prdct_key}'
    }
    task=ee.batch.Export.image.toCloudStorage(**export_config)
    task.start()

In [136]:
##download all the related products
dict_prdct={
'mean_2m_air_temperature': 'Daily mean 2m air temperature',
'total_precipitation': 'Daily total precipitation sums',
'dewpoint_2m_temperature': "Daily mean 2m dewpoint temperature",
'mean_sea_level_pressure':"Daily mean sea-level pressure",
'surface_pressure': "Daily mean surface pressure",
'u_component_of_wind_10m': "Daily mean 10m u-component of wind",
}
for prd in dict_prdct:
    era5_dat = era5.select(prd)
    print('Number of images', era5_dat.size().getInfo())
    export_ERA5_to_GCPbucket(era5_dat,aoi=aoi, bucket_name='AlgaeDrive',crs_epsg=32615,scale=1000,prdct_key=prd)

Number of images 30
Number of images 30
Number of images 30
Number of images 30
Number of images 30
Number of images 30


## 2. download satellite imagery of the same dates/AOI

In [ ]:
##download satellite imagery to bucket
def export_OLCI_to_GCPbucket(dataset,aoi=None, bucket_name='AlgaeDrive',crs_epsg=4326,scale=300):
  img_list = dataset.toList(dataset.size())
  nImages = img_list.size().getInfo()
  for i in range(nImages):
    img=ee.Image(img_list.get(i)).clip(aoi)
    product_id=img.get('PRODUCT_ID').getInfo()
    export_config = {
      'image': img.float(),
      'region': aoi,
      'crs': f'EPSG:{crs_epsg}',
      'scale': scale,    #default in 300 pixels of OLCI
      'fileFormat': 'GeoTIFF',
      'fileNamePrefix': f'OLCI/{product_id}',
      'bucket':   bucket_name,  
      'maxPixels':1e10,
      'description': 'ESA OLCI ocean color satellite imagery'
    }
    task=ee.batch.Export.image.toCloudStorage(**export_config)
    task.start()

crs_epsg=32615  #the epsg code for projection
start_date,end_date='2020-06-01','2020-07-01'
aoi =ee.Geometry.Rectangle(-95.4, 48.88,-93.75, 49.77)
sensor='COPERNICUS/S3/OLCI'
imgs=ee.ImageCollection(sensor) \
       .filterBounds(aoi)  \
       .filterDate(ee.Date('2022-09-01'), ee.Date('2022-09-03'))
export_OLCI_to_GCPbucket(imgs,aoi=aoi, bucket_name='AlgaeDrive',crs_epsg=4326,scale=300)